In [53]:
import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
from metocean_api import ts
import tqdm

# Path to your local NetCDF file
file_path = "/Users/janniskerl/Downloads/TU-MREL_EU_ATL-2M_202101.nc"

# Load dataset (now fully local – no network)
dataset = xr.open_dataset(file_path)


In [ ]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator
import pandas as pd

# Define Tp and Hm0 axes from your matrix, 700 kW Matrix
Tp_values = np.array([5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0])
Hm0_values = np.array([0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0,6.5,7.0])

# Your power matrix (kW)
power_matrix_values = np.array([
    [0,0,20,30,40,43,45,45,38,35,30,25],
    [0,10,30,50,65,70,75,80,70,53,50,48],
    [0,35,63,90,110,125,140,140,130,100,80,75],
    [10,50,88,120,160,180,185,190,180,125,115,105],
    [27,80,130,190,225,230,235,240,210,190,165,130],
    [45,115,170,240,260,300,310,320,250,217,180,160],
    [60,145,230,300,320,330,345,350,300,245,220,190],
    [75,160,300,340,365,400,400,400,340,300,240,225],
    [85,180,340,380,400,445,440,450,395,320,280,250],
    [95,230,380,455,480,490,500,510,440,350,300,280],
    [125,250,440,500,530,560,560,570,490,390,330,310],
    [140,300,460,570,600,610,600,600,500,460,390,340],
    [150,350,530,630,650,690,680,700,580,540,400,370],
    [175,350,560,690,700,750,750,750,650,540,440,400]
])

# Interpolator: for continuous Tp/Hm0 input
power_interpolator = RegularGridInterpolator(
    (Hm0_values, Tp_values),
    power_matrix_values,
    bounds_error=False,
    fill_value=0.0
)

In [55]:
# Load EU NUTS2 data (excluding GB)
nuts2_eu_path = 'NUTS_RG_60M_2024_3035.geojson'
nuts2_eu_gdf = gpd.read_file(nuts2_eu_path)
nuts2_eu_gdf = nuts2_eu_gdf[nuts2_eu_gdf['LEVL_CODE'] == 2]  # Keep only NUTS2 regions

# Load GB-specific NUTS2 data
nuts2_gb_path = 'NUTS_RG_60M_2021_3035.geojson'
nuts2_gb_gdf = gpd.read_file(nuts2_gb_path)
nuts2_gb_gdf = nuts2_gb_gdf[nuts2_gb_gdf['LEVL_CODE'] == 2]  # Keep only NUTS2 regions 
nuts2_gb_gdf = nuts2_gb_gdf[nuts2_gb_gdf['CNTR_CODE'] == 'UK']

# Merge EU and GB NUTS2 regions
nuts2_gdf = pd.concat([nuts2_eu_gdf, nuts2_gb_gdf], ignore_index=True)
nuts2_gdf = nuts2_gdf.drop_duplicates(subset=['NUTS_ID'])
nuts2_gdf

nuts2_gdf = nuts2_gdf.to_crs("EPSG:4326")  # Required for spatial join


In [56]:
lat_vals = dataset['latitude'].values
lon_vals = dataset['longitude'].values
lons, lats = np.meshgrid(lon_vals, lat_vals)

In [57]:
flat_points = gpd.GeoDataFrame({
    'lat': lats.ravel(),
    'lon': lons.ravel(),
    'geometry': [Point(xy) for xy in zip(lons.ravel(), lats.ravel())]
}, crs='EPSG:4326')

# Spatial join with your combined NUTS2 dataset
flat_points = gpd.sjoin(flat_points, nuts2_gdf, how='left', predicate='within')

# Keep only what you need
lookup_df = flat_points[['lat', 'lon', 'NUTS_ID']]


In [58]:
# Extract Tp (peak period), use ptp0 with fallback to ptp1
Tp0 = dataset["ptp0"]
Tp1 = dataset["ptp1"]

In [59]:
results = []

for i in range(len(dataset['time'])):
    time_step = dataset['time'].isel(time=i).values

    # 1. Load wave height (Hs) and peak period (Tp or dp) for current timestep
    hs = dataset['hs'].isel(time=i).values  # shape: (lat, lon)
    tp = dataset['dp'].isel(time=i).values  # shape: (lat, lon)
    # Get shape info
    lat_len = hs.shape[0]
    lon_len = hs.shape[1]

    # Flatten hs and tp for current time step
    hs_flat = hs.ravel()
    tp_flat = tp.ravel()

    # Create a per-timestep version of lookup_df
    lookup_df_current = lookup_df.iloc[:hs_flat.shape[0]].copy()  # <- aligns length

    # Valid mask
    valid_mask = ~np.isnan(hs_flat) & ~np.isnan(tp_flat)

    # Filter everything consistently
    hs_valid = hs_flat[valid_mask]
    tp_valid = tp_flat[valid_mask]
    grid_subset = lookup_df_current[valid_mask].copy()  # aligned mask

    if len(hs_valid) == 0:
        continue  # No data to process for this timestep

    # 3. Interpolate power values from the power matrix
    points = np.stack([hs_valid, tp_valid], axis=1)
    power_values = power_interpolator(points)

    # 4. Add interpolated power and timestamp to the grid
    grid_subset['power_kw'] = power_values
    grid_subset['time'] = time_step

    # 5. Aggregate total power per NUTS region
    grouped = (
        grid_subset.groupby('NUTS_ID')['power_kw']
        .sum()
        .reset_index()
    )
    grouped['time'] = pd.to_datetime(time_step)

    results.append(grouped)


In [60]:
results

[   NUTS_ID      power_kw       time
 0     BE25      0.000000 2021-01-01
 1     DE94      0.000000 2021-01-01
 2     ES11    158.180002 2021-01-01
 3     ES12      0.000000 2021-01-01
 4     ES13    289.220018 2021-01-01
 5     ES21      0.000000 2021-01-01
 6     FRD1      0.000000 2021-01-01
 7     FRD2      0.000000 2021-01-01
 8     FRE2      0.000000 2021-01-01
 9     FRG0      0.000000 2021-01-01
 10    FRH0      0.000000 2021-01-01
 11    FRI1      0.000000 2021-01-01
 12    FRI3      0.000000 2021-01-01
 13    IE04     27.288002 2021-01-01
 14    IE05     40.224003 2021-01-01
 15    IE06      0.000000 2021-01-01
 16    NL11      0.000000 2021-01-01
 17    NL12      0.000000 2021-01-01
 18    NL32      0.000000 2021-01-01
 19    NL34      0.000000 2021-01-01
 20    NL36      0.000000 2021-01-01
 21    PT11      0.000000 2021-01-01
 22    PT19      0.000000 2021-01-01
 23    UKC1   1361.128060 2021-01-01
 24    UKC2    528.772039 2021-01-01
 25    UKD1      0.000000 2021-01-01
 

In [61]:
print(dataset['time'].isel(time=0).values)

2021-01-01T00:00:00.000000000
